In [1]:
import os
import re
import pandas as pd
import datetime
from datetime import timedelta
from dateutil.parser import parse

from multiprocessing import Process, Manager, Pool, cpu_count
from collections import Counter

from proc_titles_function import my_function_wrapper

# how many days after the end of a natural disaster to consider an article hit
days_to_add = 60
days_to_sub = 50
random_year = 2025

# buckets for year quarters
quarter_starts = []
quarter_starts.append(datetime.datetime(random_year, 1, 1))
quarter_starts.append(datetime.datetime(random_year, 4, 1))
quarter_starts.append(datetime.datetime(random_year, 7, 1))
quarter_starts.append(datetime.datetime(random_year, 10, 1))
quarter_starts.append(datetime.datetime(random_year, 12, 31))

##

# notes:

# bbc needs to have custom scraper for news titles
# bloomberg: a few in 3rd index of split /
# cnbc: .html in link
# daily mail: need to make sure that there is a date for the articles, same as cnbc
# huffpost: TBD
# foxnews: done
# nbc: NEED TO GET TITLES FOR "ID" ARTICLES

# daily star?


In [2]:

path_to_news = "/Users/tristanbrigham/Desktop/Self Study/Computer Science/News Scraper/News Sites Folders/"
path_to_article_titles = "/Users/tristanbrigham/Desktop/test_news/Articles_titles"
col_headers = ["link", "date"]

# format: 
# headline delimit character
# idx of check phrase
# interesting words (spaced by '|') -get_article_links- if this is empty then take all
# date format of link if it exists
link_dict = {
    "bloomberg" : ('-', ["opinion", "news", "en", "features", "politics"], "y-m-d") ,
    "cnbc" : ('-', ["html"], "y/m/d") ,
    "dailymail" :  ('-', ["html"], "") ,
    "foxnews" :  ("-", ["weather", "area-51", "official-polls", "austin", "apps-products-ios", "tech", "fox-and-friends", "family", "fox-friends", "faith-values", "great-outdoors", "slack-success", "health", "lifestyle", "miami", "midterms-2018", "slack-app", "nyc", "newsletters", "about", "compliance", "auto", "politics", "story", "donotsell", "shows", "food-drink", "real-estate", "travel", "updates", "gowatch", "recordkeeping", "transcript", "cw-the-real-texas-rangers", "instant-access", "person", "whats-changed", "newsletters-coronavirus", "chicago", "opinion", "science", "sports", "uncategorized", "sunday-morning-futures", "world", "fox-news", "story", "us", "entertainment"], "") ,
    "nypost" : ("-", '', "y/m/d") ,
    "thedailybeast" : ('-', "", "") ,
    "theglobeandmail" : ("-", "", "") ,
    "usatoday" : ('-', "", 'y/m/d') ,
    "cnn" :  ('-', ["html"], 'y/m/d') ,
    "bbc" :  ('_', "", 'y/m/d') ,
    "nytimes" : ('-', ['html'], "y/m/d") ,
    "washingtonpost" : ('-', '', "") ,
    "dailystar" : ('-', '', "") ,
    "huffpost" : ('-', ["entry"], "") ,
    

    # fix id issue
    # "nbcnews" : ("-", 3, '/', ["news", "globalcitizen", "think", "dateline", "feature", "health", "investigations", "meet-the-press", "nbc-out", "nightly-news", "politics", "pop-culture", "science", "space", "tech", "tornadoes", "western-wildfires", "world", "msnbc", "media", "data-graphics", "author", "slideshow", "information", "storyline", "nbc-learn", "businessmain", "healthmain", "_news", "sciencemain", "technolog", "test", "usnews", "worldmain",] "") ,

    
    # perma-block
    # "usnews" : ('-', 0, '', "") ,
    # "cbsnews" :  ('_', -1, '.', "news") ,
}



path_to_news = "/Users/tristanbrigham/Desktop/Self Study/Computer Science/News Scraper/News Sites Folders/"
path_to_article_titles = "/Users/tristanbrigham/Desktop/Citadel Datathon/Articles_titles/"
col_headers = ["link", "date"]

# define a function to open the csv and then return the dataframe
def get_article_links(name):

    links_df = pd.read_csv(path_to_news + name + "/news_articles.csv", header=None, index_col=False)
    links_df.columns = col_headers

    # print(links_df.head(5))

    return links_df


# this is meant to parse the link and try to find a date in the case that the article doesn't have one
def get_date(link, args):

    try:

        date_format = args[3]

        if date_format == "":

            return ""
        
        # now we can assume that there is a date format
        # search the link for a date (only considering 2000 and beyond)
        date_idx = link.index("20")

        # check if this is a random 20 or not
        if link[date_idx + 4 != args[3][1]]:

            return ""

        # assume that this is not a random 20
        date = link[date_idx : date_idx + 10]

        return date

    except:

        # print(f"Couldn't find date: {link}")
        return ""
    

# this function takes the title and splits it based off of the arguments provided
def split_title(link, args):

    out = link.split(args[0])

    out = " ".join(out).replace("https://www.cnbc.com/", "").replace(".html", "").replace("https://www.bloomberg.com/", "").replace("https://www.theglobeandmail.com/", "").replace("https://www.thedailybeast.com/", "").replace("https://nypost.com/", "").replace("https://www.foxnews.com/", "").replace("https://www.bbc.com/", "").replace("https://www.dailymail.co.uk/", "").replace("edition.", "").replace("https://cnn.com/", "").replace("https://www.usatoday.com/", "").replace("https://www.huffingtonpost.com/", "").replace("https://www.washingtonpost.com/", "").replace("https://www.nytimes.com/", "").replace("https://www.dailystar.co.uk/", "")

    out = out.split("/")

    return " ".join(out)


def count_words_in_string(word_array, target_string):
    common_words = [word for word in word_array if word in target_string]
    return len(common_words)


# this function checks whether the title of the news article that it is passed has any of the words that might indicate weather event
# this is done in two ways -- first, we are looking for any words that are in the list that we have passed it below
# second, we are looking for the names of any storms 
def check_weather(link):

    weather_words = ['storm', 'hurricane', 'typhoon', 'tornado', 'cyclone', 'thunderstorm', 'flood', 'drought', 
    'heat', 'cold', 'snowstorm', 'blizzard', 'hailstorm', 'monsoon', 'rainfall', 'snowfall', 'temperature', 
    'earthquake', 'tsunami', 'volcano', 'wildfire', 'landslide', 'avalanche', 'eruption', 'seismic', 'environment', 
    'climate change', 'biodiversity', 'deforestation', 'ozone', 'sunny', 'cloudy',
    'windy', 'rainy', 'snowy', 'overcast', 'humid', 'foggy', 'aurora', 'eclipse', 'meteor', 'comet', 'solar', 
    'flare', 'lightning', 'precipitat', 'drizzle', 'sleet', 'muggy', 'breezy', 'gust', 'heat', 'wind', 'dew', 
    'barometer', 'thermometer', 'anemometer', 'forecast', 'atmospheric', 'pressure', 'uv', 'index', 
    'seismology', 'aftershock', 'fault line', 'evacuation', 'emergency', 'disaster', 'relief', 'catastrophe', 
    'resilience', 'preparedness', 'shelter', 'disaster recovery', 'warm front', 'cold front', 'weather system', 
    'isobar', 'microclimate', 'atmospheric conditions', 'weather patterns', 'climate patterns', 
    'aurora borealis', 'northern lights', 'aurora australis', 'southern lights', 'meteor shower', 
    'solar', 'eclipse', 'lunar', 'eclipse', 'gravity wave', 'cosmic', 'rays', 'coronal mass ejection', 'snowfall', 
    'snowstorm', 'blizzard', 'snowdrift', 'snowflake', 'frost', 'ice', 'snowbank', 'slush', 'snowmelt', 
    'hail', 'winter', 'frosty', 'winter wonderland', 'polar vortex', 'snow', 'snow-capped', 'avalanche', 'whiteout', 
    'rainfall', 'showers', 'downpour', 'drizzle', 'cloudburst', 'deluge', 'rainstorm', 'raindrops', 'precipitation', 
    'thunderstorm', 'monsoon', 'rainy', 'wet', 'rain', 'rain gauge', 'rain cloud', 'drenching rain', 'rain shower', 
    'rainy season', 'rain boots', 'raincoat', 'windy', 'breezy', 'gust', 'gale', 'zephyr', 'bluster', 'whirlwind', 
    'cyclone', 'tornado', 'twister', 'tempest', 'windstorm', 'wind chill', 'windy weather', 'anemometer', 'wind', 
    'wind speed', 'wind', 'farm', 'wind', 'energy', 'system', 'pattern', 'atmospheric', 'condition', 
    'forecast', 'meteorology', 'front', 'weather']

    # now check if the name of any storms exists
    storm_words = [
        'iniki', 'opal', 'sandy', 'imelda', 'isidore', 'michael', 'hugo', 
        'katrina', 'nicholas', 'storm', 'flood', 'irma', 'matthew', 'wildfire', 'elena', 
        'ian', 'harvey', 'allison', 'georgia', 'alberto', 'ike', 'blizzard', 'floods', 'isaac', 'allen', 
        'tropical', 'hanna', 'fire', 'cold', 'dorian', 'storm', 
        'hail', 'ice', 'jeanne', 'blizzard', 'hurricane', 'dolly', 'fiona', 'laura', 'delta', 
        'maria', 'isabel', 'lee', 'midwest/ohio', 'isaias', 'kansas', 'irene', 'wave', 'england', 'storm,', 
        'drought', 'heat', 'ivan', 'charley', 'floyd', 'ida', 'drought', 'andrew', 'flooding', 'gustav', 
        'erin', 'elsa', 'dennis', 'nicole', 'zeta', 'fran', 'heat', 'lili', 'georges', 'tornado', 'midwest', 
        'fred', 'juan', 'wilma', 'derecho,', 'marilyn', 'rita', 'sally', 'bob', 'firestorm', 'eta', 'gloria', 
        'derecho', 'winter', 'wildfires', 'alicia', 'freeze', 'bonnie', 'flash'
        ]

    
    # for word in storm_words:

    #     if word.lower() in link.lower():

            # print(f"STORM: {word}")

    # print(f"storm: {any(word.lower() in link.lower() for word in storm_words)}")
    # print(f"weather: {any(word.lower() in link.lower() for word in weather_words)}")

    pattern = r"[!@#$%^&*()\-_=+[{\]};:'\",<.>/?\\|`~]"

    # Use re.sub() to replace the special characters with spaces
    link = re.sub(pattern, ' ', link)

    return any(word in [x.lower() for x in link.split(" ")] for word in storm_words) or any(word in [x.lower() for x in link.split(" ")] for word in weather_words)

        
    
    
def get_type_disaster(link):

    pattern = r"[!@#$%^&*()\-_=+[{\]};:'\",<.>/?\\|`~]"

    # Use re.sub() to replace the special characters with spaces
    link = re.sub(pattern, ' ', link)

    types_disasters = ["Drought", "Cold", "Storm", "Fire", "Flood", "Cyclone"]

    drought_dict = ['drought', 'arid', 'dry', 'thirst', 'parched', 'waterless', 'scorching', 'heatwave', 'dehydrated', 'desiccated', 'dusty', 'cracked', 'barren', 'hot', 'thirsty', 'withered', 'deficit', 'scorched', 'rainless', 'shrinking', 'severe', 'lack', 'aridity', 'desert', 'water shortage', 'scarcity', 'heat', 'crop failure', 'dryness', 'famine', 'wildfire', 'shortage', 'water crisis', 'water stress', 'water rationing', 'waterless', 'dry spell', 'dustbowl', 'crop loss', 'thirsty crops', 'parch', 'crop damage', 'drying', 'water conservation', 'rainfall deficiency', 'desertification', 'heat stress', 'reservoir levels', 'water depletion', 'hydrological drought']
    cold_storm_dict = ['snowstorm', 'blizzard', 'frosty', 'chills', 'icy', 'snowy', 'brr', 'snowflakes', 'freeze', 'flurries', 'sleet', 'slush', 'frost', 'icicles', 'polar', 'arctic', 'whiteout', 'cold', 'wintry', 'snowfall', 'winter', 'snowbound', 'snowpack', 'hail', 'drifts', 'freeze', 'winterize', 'snowmageddon', 'snowbanks', 'chill', 'frostbite', 'snowslide', 'hibernation', 'frigid', 'snowcap', 'snowmobile', 'snowshoes', 'snowplow', 'snowman', 'avalanche', 'powder', 'slippery', 'shiver', 'iciness', 'snowscape', 'snowbelt', 'snowball', 'winterize', 'snowy', 'cold snap']
    other_storm_dict = ["Storm", "Thunder", "Rain", "Flash", "Hail", "Wind", "Flood", "Clouds", "Lightning", "Snow", "Sleet", "Tornado", "Gales", "Downpour", "Squall", "Deluge", "Blustery", "Cyclone", "Tempest", "Drizzle", "Monsoon", "Cloudburst", "Whirlwind", "Typhoon", "Hurricane", "Drench", "Shower", "Torrent", "Wet", "Thunderclap", "Rainfall", "Snowfall", "Stormy", "Gusts", "Thunderhead", "Lightning strike", "Rumble", "Frost", "Mist", "Polar", "Gale-force", "Fog", "Twister", "Blizzard", "Snowflake", "Sleet", "Wetness", "Lightning bolt", "Flooded", "Cyclonic",  ]
    fire_dict = ['fire', 'blaze', 'inferno', 'flame', 'wildfire', 'burn', 'hot', 'smoke', 'heat', 'combustion', 'charred', 'ash', 'ember', 'scorch', 'smolder', 'ignite', 'flare', 'sear', 'scorched', 'incinerate', 'pyre', 'kindle', 'singe', 'conflagration', 'heatwave', 'flaming', 'fiery', 'spark', 'sizzle', 'blaze up', 'firestorm', 'burning', 'flameout', 'bonfire', 'incendiarism', 'flashfire', 'infernal', 'engulf', 'arson', 'firefighting', 'firefighter', 'blazing', 'ignition', 'fireball', 'fireproof', 'smoky', 'cinders', 'firetruck', 'combustible', 'flamingo']
    flood_dict = ['flood', 'deluge', 'inundation', 'water', 'overflow', 'surge', 'submerge', 'swell', 'torrent', 'overflowing', 'rains', 'flash flood', 'waterlogged', 'dam', 'flooding', 'soaked', 'river', 'flooded', 'rising', 'evacuation', 'disaster', 'reservoir', 'emergency', 'levee', 'submerged', 'torrential', 'waterway', 'floodplain', 'floody', 'rainfall', 'floodwater', 'leaking', 'damaged', 'damaging', 'barrier', 'riverbank', 'inflow', 'deluged', 'floodgate', 'overflowed', 'flooded-out', 'rainstorm', 'water surge', 'overtop', 'soggy', 'flood-prone', 'heavy rain', 'flooded area', 'waterlogged', 'rainfall records']
    cyclone_typhoon_dict = ['cyclone', 'typhoon', 'storm', 'wind', 'tropical', 'hurricane', 'gales', 'weather', 'dangerous', 'destruction', 'devastating', 'powerful', 'cyclonic', 'eye', 'landfall', 'intense', 'category', 'rain', 'evacuation', 'threat', 'warning', 'disaster', 'windy', 'severe', 'tornadoes', 'coastal', 'windswept', 'shelter', 'emergency', 'damaging', 'impact', 'prepare', 'safety', 'damage', 'landfall', 'surge', 'relief', 'resilience', 'evacuate', 'precautions', 'preparedness', 'cyclonic storm', 'typhoon warning', 'evacuation plan', 'weather alert', 'severe weather', 'cyclone aftermath', 'emergency response', 'tropical storm', "cyclone's path"]

    temp_title = link
    score_array = []

    for l in [drought_dict, cold_storm_dict, other_storm_dict, fire_dict, flood_dict, cyclone_typhoon_dict]:

        score_array.append(count_words_in_string(l, temp_title))


    max_value = max(score_array)

    if max_value == 0:

        type_disaster = "unknown"

    else:
        
        type_disaster = types_disasters[score_array.index(max_value)]

    return type_disaster



def split_date(date):

    date = str(date)

    year = date[:4]
    month = date[4:6]
    day = date[6:]

    return str(year) + "/" + str(month) + "/" + str(day)




# checking if this date is in the right quarter (year agnostic)
def check_quarter(date, idx):

    return quarter_starts[idx] <= date.replace(year=random_year) <= quarter_starts[idx + 1]

In [3]:
# create a new df for weather event mentions and then add that to the base one

# def process_titles(weather_event_df):

#     for key, information in link_dict.iterrows():
    
#         # key, information = next(iter(link_dict.items()))

#         # go through each of the pairings... get the name of the news org and the respective news data first
#         temp_df = get_article_links(key)

#         # check if the right text is in the article title
#         if information[2] != "":
            
#             # drop any of the rows that don't contain right text
#             temp_df = temp_df[temp_df['link'].str.contains('|'.join(information[2]))]

#         # now that we have the associated dataframe, go link by link and process it into individual words 
#         # that we can compare against our bad word dictionary
#         # make sure that we keep the date there too

#         # remember, we still need to process nbcnews'

#         # temp_df["date"] = temp_df["date"].apply(lambda row: get_date(row["link"], information) if row["date"].isnull().all() else row)

#         for _, row in temp_df.iterrows():

#             # check if there is a date attached
#             if pd.isna(row["date"]):

#                 row["date"] = get_date(row["link"], information)
            
#             # now we either have a date or we don't
#             # if we don't, we can't consider this article link and should drop it

#         temp_df = temp_df.dropna(subset=["date"])
#         temp_df = temp_df.loc[~(temp_df["date"] == "")]

#         ## WEATHER STUFF
#         # check if the article mentions any weather events
#         temp_df["mentions_weather"] = temp_df["link"].apply(check_weather)
#         temp_df["weather_type"] = None
#         temp_df["weather_type"] = temp_df[temp_df["mentions_weather"]]["link"].apply(get_type_disaster)

            
#         # create a new df column that has the article names
#         temp_df["title"] = temp_df["link"].apply(lambda age: split_title(age, information))


#         # go line by line in the df now and increase the reference count for news-worthy weather events
#         for _, men_row in temp_df[temp_df["mentions_weather"] == True].iterrows():

#             # parse the date of the article
#             article_date = parse(men_row["date"]).replace(tzinfo=None)

#             # now add one to the reference count for weather events
#             # for _, w_row in weather_event_df.iterrows():
                
#             weather_event_df.loc[(weather_event_df["start"] < article_date) & (weather_event_df["end"] + timedelta(days=days_to_add) > article_date), "mentions"] += 1
#                 # weather_event_df[parse(weather_event_df["start"]) < article_date & parse(weather_event_df["end"]) + timedelta(days=days_to_add) > article_date, "mentions"] += 1
            

#         # save this to a new csv in my local citadel correlation folder
#         temp_df.to_csv(path_to_article_titles + key + "_titles.csv", index=False)


#         print(f"DONE: {key}")
    
#     return weather_event_df


In [4]:
# now we are going to go through each of the links in each of the news organzations
# and parse the title of the article out from each of them

# note that nbc news has a lot of titles that have "id wbna" in the title, so we are going to try to request those

# get the path to the disaster data:
path_to_disaster_data = "/Users/tristanbrigham/Desktop/Citadel Datathon/Citadel_Correlation_One_Datathon/Auxillary Data/NOAA Disaster Data/disasters_NOAA_events-US-1980-2023.csv"

# we are going to create a dataframe of storms now from the csv file that we have
weather_event_df = pd.read_csv(path_to_disaster_data)

# # changing the column names so that it is more intuitive
# new_column_names = {"Name"  : 'name',
#                     "Disaster" : 'disaster_type',
#                     "Begin Date" : 'start',
#                     "End Date" : 'end',
#                     "Total CPI-Adjusted Cost (Millions of Dollars)" : 'cost',
#                     "Deaths" : 'deaths'}

# rename the columns
# weather_event_df.rename(columns=new_column_names, inplace=True)

# fix the data data so that it is reflective
weather_event_df['start'] = weather_event_df['start'].apply(lambda x: parse(x).replace(tzinfo=None))
weather_event_df['end'] = weather_event_df['end'].apply(lambda x: parse(x).replace(tzinfo=None))

weather_event_df['mentions'] = 0

print(weather_event_df.head(5))


# now we have our weather data set up

                                                name     disaster_type   
0   Southern Severe Storms and Flooding (April 1980)          Flooding  \
1                      Hurricane Allen (August 1980)  Tropical Cyclone   
2  Central/Eastern Drought/Heat Wave (Summer-Fall...           Drought   
3                      Florida Freeze (January 1981)            Freeze   
4  Severe Storms, Flash Floods, Hail, Tornadoes (...      Severe Storm   

       start        end     cost  deaths  mentions     Q1     Q2     Q3   
0 1980-04-10 1980-04-17   2650.4       7         0  False   True  False  \
1 1980-08-07 1980-08-11   2153.6      13         0  False  False   True   
2 1980-06-01 1980-11-30  39178.2    1260         0  False   True  False   
3 1981-01-12 1981-01-14   2002.0       0         0   True  False  False   
4 1981-05-05 1981-05-10   1360.8      20         0  False   True  False   

      Q4         str          nd  
0  False  1980-02-20  1980-06-16  
1  False  1980-06-18  1980-10-10  

In [5]:
num_processes = cpu_count()  # Number of CPU cores available

input_data = []
results = []
    
for key, info in link_dict.items():

    input_data.append((key, info, weather_event_df.copy()))

# Create a pool of processes with the desired number of worker processes
with Pool(processes=num_processes) as pool:

    # Use the DataFrame rows as the iterable for parallel processing
    results.append(pool.map(my_function_wrapper, input_data))

# Update the 'Result' column with the output of the function
for df in results:

    weather_event_df["mentions"] += df["mentions"]

print(weather_event_df)


STARTING: bloomberg
STARTING: dailymail
STARTING: cnbc
STARTING: thedailybeast
STARTING: foxnews
STARTING: usatoday
STARTING: nypost
STARTING: theglobeandmail
STARTING: nytimes
STARTING: cnn
STARTING: bbc
STARTING: washingtonpost
STARTING: dailystar
STARTING: huffpost


KeyboardInterrupt: 

In [ ]:

weather_event_df["Q1"] = weather_event_df["start"].apply((lambda date: check_quarter(date, 0))) | weather_event_df["end"].apply(lambda date: check_quarter(date, 0))
weather_event_df["Q2"] = weather_event_df["start"].apply((lambda date: check_quarter(date, 1))) | weather_event_df["end"].apply(lambda date: check_quarter(date, 1))
weather_event_df["Q3"] = weather_event_df["start"].apply((lambda date: check_quarter(date, 2))) | weather_event_df["end"].apply(lambda date: check_quarter(date, 2))
weather_event_df["Q4"] = weather_event_df["start"].apply((lambda date: check_quarter(date, 3))) | weather_event_df["end"].apply(lambda date: check_quarter(date, 3))
# weather_event_df["Q1"] = weather_event_df["start"].apply((lambda date: check_quarter(date, 0))) | weather_event_df["end"].apply(lambda date: check_quarter(date, 0))
# weather_event_df["Q2"] = weather_event_df["start"].apply((lambda date: check_quarter(date, 1))) | weather_event_df["end"].apply(lambda date: check_quarter(date, 1))
# weather_event_df["Q3"] = weather_event_df["start"].apply((lambda date: check_quarter(date, 2))) | weather_event_df["end"].apply(lambda date: check_quarter(date, 2))
# weather_event_df["Q4"] = weather_event_df["start"].apply((lambda date: check_quarter(date, 3))) | weather_event_df["end"].apply(lambda date: check_quarter(date, 3))

# see if you can add some propogation to this? Like a flag that says prev quarter wasn't awesome
article_date = parse("2022/8/28").replace(tzinfo=None)
weather_event_df.loc[(weather_event_df["start"] - timedelta(days=days_to_sub) < article_date) & (weather_event_df["end"] + timedelta(days=days_to_add) > article_date), "mentions"] += 1
weather_event_df["str"] = weather_event_df["start"] - timedelta(days=days_to_sub)
weather_event_df["nd"] = weather_event_df["end"] + timedelta(days=days_to_add)

weather_event_df.to_csv(path_to_disaster_data, index=False)

In [ ]:
# # let's add quarters to the df
# for idx, title in temp_df[temp_df["mentions_weather"] == True].iterrows():

#     print(title["title"])
#     print(check_weather(title["link"]))

#     if idx > 500:

#         break

# temp_df.columns

In [ ]:
print(datetime.now())